In [2]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import pandas as pd

df = pd.read_csv("Training_cleaned.csv")
df.head()

C:\Users\riley\AppData\Local\Temp\ipykernel_28592\2890702488.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Unnamed: 0,Id,year,bpm,nrgy,dnce,dB,live,val,dur,...,genre_merseybeat,genre_neo mellow,genre_new wave pop,genre_permanent wave,genre_pop,genre_r&b,genre_rock-and-roll,genre_soft rock,genre_uk garage,genre_yodeling
0,0,1,1996,107,31,45,-8,13,28,150,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,2011,114,44,53,-8,13,47,139,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,1979,105,36,63,-9,13,67,245,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,1980,170,28,47,-16,13,33,232,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,1973,121,47,56,-8,15,40,193,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from sklearn.model_selection import train_test_split

x = df.drop(['pop'],axis=1)
y = df["pop"]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)


# Linear Regression 

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


reg = LinearRegression().fit(x_train, y_train)
reg.coef_
reg.intercept_

y_pred = reg.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print(np.sqrt(mse))



199241858329917.84


### Notes
- very poor mse , probably better options out there 

# Stochastic Gradient Decent Regression 

In [82]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

sgd_reg = SGDRegressor(random_state=42)
sgd_reg.fit(x_train, y_train)
sgd_reg.score(x_train,y_train)
sgd_reg.coef_
sgd_reg.intercept_

y_pred = sgd_reg.predict(x_test)
mse = mean_squared_error(y_test, y_pred)

print(np.sqrt(mse))




16547671316223.422


### Notes:
- Score is much larger than expeceted 
- Fluxuating hyperparameters doesn't seem to change the score all too much

# Non-Linear/Polynomial Regression 

In [83]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2,include_bias=False)

x_poly = poly.fit_transform(x_train)

poly.fit(x_train, y_train)

lin_poly=LinearRegression().fit(x_poly, y_train)

y_pred = lin_poly.predict(poly.fit_transform(x_test))

mse = mean_squared_error(y_test, y_pred)

print(np.sqrt(mse))



21.776477572551993


### Notes: 
- Better than SGD but worse than simple linear regression 

# Logistic Regression

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
Log_reg = LogisticRegression(max_iter=1000,random_state=42) # Not necessary to parameterise
Log_reg.fit(x_train,y_train)

y_pred = Log_reg.predict(x_test)
mse=mean_squared_error(y_test,y_pred)
print(f"MSE:{np.sqrt(mse)}")



MSE:18.72413967565417


c:\Users\riley\OneDrive - University of Strathclyde\AI and Applications\.conda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Notes 
- Better than Non-Linear however im scheptical of any improvements that can be made to it. 
- Worse than simple linear regression

# SVM Regression

In [85]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=8,random_state=42)
svm_reg.fit(x_train,y_train)
y_pred = svm_reg.predict(x_test)

mse=mean_squared_error(y_test,y_pred)
print(f"MSE:{np.sqrt(mse)}")

MSE:15.438507321414347


c:\Users\riley\OneDrive - University of Strathclyde\AI and Applications\.conda\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\riley\OneDrive - University of Strathclyde\AI and Applications\.conda\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Notes:
- Not bad , still lower than previous attempts

# Decision Tree Regression

In [92]:
from sklearn.tree import DecisionTreeRegressor

DT_reg= DecisionTreeRegressor(criterion='friedman_mse', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, random_state=42,
            splitter='best')
DT_reg.fit(x_train,y_train)

y_pred = DT_reg.predict(x_test)

mse=mean_squared_error(y_test,y_pred)
print(f"MSE:{np.sqrt(mse)}")





MSE:14.940265675734768


### Notes
- Still not the best we've seen however definitely promising with some tuning. 
- Theory & parameters are easier to understand so might be worth circling back to. 

# Hyperparameter tuning + Other Regressors 

In [7]:

df = pd.read_csv("Training_Cleaned.csv")
df.head()
x = df.drop(columns=['pop','Id'])
y = df["pop"]

x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)



from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import BayesianRidge,ElasticNet,LogisticRegression,SGDRegressor,LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR,SVR
from sklearn.preprocessing import PolynomialFeatures , StandardScaler


random_state = 42

GBR = GradientBoostingRegressor()
KR = KernelRidge()
BR = BayesianRidge()
EN = ElasticNet()
DT = DecisionTreeRegressor(criterion='friedman_mse', max_depth=20,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, random_state=random_state,
                        splitter='best')
Lin_SVR = LinearSVR(epsilon = 8 , random_state=random_state )
Poly_SVR = SVR(kernel="poly", degree=2, C=1, epsilon=0.1, gamma="scale")
Log_reg = LogisticRegression(random_state=random_state)
SGD = SGDRegressor(random_state=random_state)
Lin_reg = LinearRegression()

reg_list = [GBR,KR,BR,DT, Lin_SVR, Lin_reg, Log_reg, SGD,Poly_SVR]

std_scaler = StandardScaler()
x_train = std_scaler.fit_transform(x_train)
x_test = std_scaler.fit_transform(x_test)

def use_regressor(reg):
    regressor = reg.fit(x_train,y_train)
    y_pred = regressor.predict(x_test)
    mse=mean_squared_error(y_test,y_pred)
    print(f"MSE:{np.sqrt(mse)} for {reg.__class__.__name__}")
    return y_pred

for reg in reg_list:
    use_regressor(reg) 




MSE:10.957591025798855 for GradientBoostingRegressor
MSE:60.2713472855601 for KernelRidge
MSE:12.586950045893003 for BayesianRidge
MSE:16.72137541006707 for DecisionTreeRegressor
MSE:12.415226644397654 for LinearSVR


c:\Users\riley\OneDrive - University of Strathclyde\AI and Applications\.conda\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


MSE:41835074403729.23 for LinearRegression
MSE:15.361933829730258 for LogisticRegression
MSE:12.85908818062765 for SGDRegressor
MSE:15.223828938752092 for SVR


In [29]:
df = pd.read_csv("Training_Cleaned.csv")
df.head()
x_train = df.drop(columns=['pop'])
y_train = df["pop"]
x_test = pd.read_csv("Testing_Cleaned.csv")



In [30]:

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import BayesianRidge,ElasticNet,LogisticRegression,SGDRegressor,LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR,SVR
from sklearn.preprocessing import PolynomialFeatures , StandardScaler



GBR = GradientBoostingRegressor()
KR = KernelRidge()
BR = BayesianRidge()
EN = ElasticNet()
DT = DecisionTreeRegressor(criterion='friedman_mse', max_depth=20,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, random_state=random_state,
                        splitter='best')
Lin_SVR = LinearSVR(epsilon = 8 , random_state=random_state )
Poly_SVR = SVR(kernel="poly", degree=2, C=1, epsilon=0.1, gamma="scale")
Log_reg = LogisticRegression(random_state=random_state)
SGD = SGDRegressor(random_state=random_state)
Lin_reg = LinearRegression()


reg_list = [GBR,KR,BR,DT, Lin_SVR, Lin_reg, Log_reg, SGD,Poly_SVR]


In [31]:
for reg in reg_list:
    regressor = reg.fit(x_train,y_train)
    y_pred = regressor.predict(x_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- top genre_alternative country
- top genre_appalachian folk
- top genre_beach music
- top genre_big beat
- top genre_classic country pop
- ...
Feature names seen at fit time, yet now missing:
- top genre_acoustic blues
- top genre_afrobeat
- top genre_afropop
- top genre_alternative rock
- top genre_american folk revival
- ...
